In [1]:
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import TimeSeriesSplit
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn import preprocessing
from sklearn.impute import SimpleImputer
from itertools import filterfalse
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import loguniform
import plotly.express as px

In c:\Users\Jason\anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The text.latex.preview rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In c:\Users\Jason\anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The mathtext.fallback_to_cm rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In c:\Users\Jason\anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: Support for setting the 'mathtext.fallback_to_cm' rcParam is deprecated since 3.3 and will be removed two minor releases later; use 'mathtext.fallback : 'cm' instead.
In c:\Users\Jason\anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The validate_bool_maybe_none function was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In c:\Users\Jason\anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mpl

# This Script Takes in Fed Minutes Data and applies it to Logistic Regression, Random Forest and XGBoost to predict Bull or Bear Markets

#### Disabling warnings below only to supress warnings from invalid gridsearch combinations that yield no score in Logistic Regression

In [2]:
import warnings
warnings.filterwarnings("ignore")

# Read in DataFrame

In [4]:
df =pd.read_csv('Excel_data/FED_window_df.csv') # generated from window_extraction_v2. This is training and test data 
df =df.drop(columns=['Unnamed: 0']) # adds a column on import index false wont remove it
column_list = df.columns.tolist()


# Choose the NLP Metrics to Test 


Pick any combination from the list below as input parameters

In [5]:
#NLP_data = ['cosine_sim', 'bertopic_num', 'LDA_1', 'LDA_2', 'LDA_3', 'LDA_4',
#       'LDA_5', 'LDA_6', 'LDA_7', 'LDA_8', 'LDA_9', 'LDA_max_value', 'w2v_1',
#       'w2v_2', 'w2v_3', 'w2v_4', 'w2v_5', 'w2v_mean', 'd2v_1', 'd2v_2',
#       'd2v_3', 'd2v_4', 'd2v_5', 'd2v_6', 'd2v_7', 'd2v_8', 'd2v_9', 'd2v_10',
#       'd2v_mean', 'd2v_umap_first_component', 'd2v_umap_second_component',
#       'Sent_TFMR','USE_Score']

## Input Chosen Parameters into the cell below

In [6]:
NLP_metric = ['d2v_umap_first_component', 'd2v_umap_second_component', 'USE_Score','bertopic_num']


# Pre built lists for parsing purposes

In [7]:
df_data = ['XLE_Adj Close', 'XLU_Adj Close', 'XLK_Adj Close', 'XLB_Adj Close', 'XLP_Adj Close', 
       'XLY_Adj Close', 'XLI_Adj Close', 'XLC_Adj Close', 'XLV_Adj Close', 'XLF_Adj Close', 
       'XLRE_Adj Close', 'HYBL_Adj Close', 'SJNK_Adj Close', 'SPTI_Adj Close', 'cosine_sim', 'bertopic_num', 'LDA_1', 'LDA_2', 'LDA_3', 
       'LDA_4', 'LDA_5', 'LDA_6', 'LDA_7', 'LDA_8', 'LDA_9', 'LDA_max_value', 'w2v_1', 'w2v_2', 'w2v_3', 'w2v_4', 'w2v_5', 
       'w2v_mean', 'd2v_1', 'd2v_2', 'd2v_3', 'd2v_4', 'd2v_5', 'd2v_6', 'd2v_7', 'd2v_8', 'd2v_9', 'd2v_10', 'd2v_mean', 
       'd2v_umap_first_component', 'd2v_umap_second_component', 'USE_Score', 'Fisher_left', 'Plosser_left', 'Bernanke_left',
        'Geithner_left', 'Yellen_left', 'Associate Director_left', 'Brexit_left', 'Jerome H. Powell_left', 'John C. Williams_left', 
        'Raphael W. Bostic_left', 'SecretaryMr_left', 'Lockhart_left', 'Cumming_left', 'GSE_left', 'KrosznerMs_left', 'PianaltoMr_left',
         'Associate Directors_left', 'MMIFF_left', 'MBS_left', 'Lacker_left', 'B. Any_left', 'Dudley_left', 'Williams_left', 'Kohn_left', 
         'Sack_left', 'Bid_left', 'ChairmanMr_left', 'Duke_left', 'William C. Dudley_left', 'James Bullard_left', 'Thomas M. Hoenig_left',
          'Hoenig_left', 'Ben Bernanke_left', 'Elizabeth Duke_left', 'Charles L. Evans_left', 'Richard W. Fisher_left', 'Sandra Pianalto_left', 
          'Daniel K. Tarullo_left', 'William B. English_left', 'Charles I. Plosser_left', 'Janet L. Yellen_left', 'Loretta J. Mester_left',
           'Narayana Kocherlakota_left', 'Kocherlakota_left', 'Senior Adviser_left', 'Stanley Fischer_left', 'Brian F. Madigan_left',
            'Patrick Harker_left', 'Tools_left', 'Mester_left', 'Counsel_left', 'Jeffrey M. Lacker_left', 'Kashkari_left', 
            'Mark L. Mullinix_left', 'Randal K. Quarles_left', 'Hurricanes Harvey_left', 'Irma_left', 'Maria_left', 'James A. Clouse_left', 
            'Esther L. George_left', 'Robert S. Kaplan_left', 'Neel Kashkari_left', 'YCT_left', 'FIMA_left', 'Thomas_left', 
            'Thomas Laubach_left', 'Chair Powell_left', 'COVID-19_left', 'Christopher J. Waller_left', 'Lisa D. Cook_left', 
            'Austan D. Goolsbee_left', 'Lorie K. Logan_left', 'Sent_TFMR', 'Fisher_right', 'Plosser_right', 'Bernanke_right', 
            'Geithner_right', 'Yellen_right', 'Associate Director_right', 'Brexit_right', 'Jerome H. Powell_right', 
            'John C. Williams_right', 'Raphael W. Bostic_right', 'SecretaryMr_right', 'Lockhart_right', 'Cumming_right', 'GSE_right',
             'KrosznerMs_right', 'PianaltoMr_right', 'Associate Directors_right', 'MMIFF_right', 'MBS_right', 'Lacker_right',
              'B. Any_right', 'Dudley_right', 'Williams_right', 'Kohn_right', 'Sack_right', 'Bid_right', 'ChairmanMr_right', 
              'Duke_right', 'William C. Dudley_right', 'James Bullard_right', 'Thomas M. Hoenig_right', 'Hoenig_right', 
              'Ben Bernanke_right', 'Elizabeth Duke_right', 'Charles L. Evans_right', 'Richard W. Fisher_right', 'Sandra Pianalto_right',
               'Daniel K. Tarullo_right', 'William B. English_right', 'Charles I. Plosser_right', 'Janet L. Yellen_right', 
               'Loretta J. Mester_right', 'Narayana Kocherlakota_right', 'Kocherlakota_right', 'Senior Adviser_right', 
               'Stanley Fischer_right', 'Brian F. Madigan_right', 'Patrick Harker_right', 'Tools_right', 'Mester_right', 
               'Counsel_right', 'Jeffrey M. Lacker_right', 'Kashkari_right', 'Mark L. Mullinix_right', 'Randal K. Quarles_right', 
               'Hurricanes Harvey_right', 'Irma_right', 'Maria_right', 'James A. Clouse_right', 'Esther L. George_right', 
               'Robert S. Kaplan_right', 'Neel Kashkari_right', 'YCT_right', 'FIMA_right', 'Thomas_right', 'Thomas Laubach_right', 
               'Chair Powell_right', 'COVID-19_right', 'Christopher J. Waller_right', 'Lisa D. Cook_right', 'Austan D. Goolsbee_right',
                'Lorie K. Logan_right', 'Return_Next_FOMC XLE', 'Return_Next_FOMC XLU', 'Return_Next_FOMC XLK', 
                'Return_Next_FOMC XLB', 'Return_Next_FOMC XLP', 'Return_Next_FOMC XLY', 'Return_Next_FOMC XLI', 'Return_Next_FOMC XLC',
                 'Return_Next_FOMC XLV', 'Return_Next_FOMC XLF', 'Return_Next_FOMC XLRE', 'Return_Next_FOMC TLT', 'Return_Next_FOMC HYBL',
                  'Return_Next_FOMC SJNK', 'Return_Next_FOMC SPTI']



NLP_testing_list = list(filterfalse(NLP_metric.__contains__,df_data)) # this removes all other NLP columns except for the desired test cases

df = df.loc[:, ~df.columns.isin(NLP_testing_list)]


Print out list to verify the desired NLP columns are included


In [8]:
df.columns.tolist()

['Date',
 'SPY_Adj Close',
 'SPY_EMA',
 'SPY_RSI',
 'SPY_MACD',
 'SPY_Signal',
 'SPY_MACD_minus_signal',
 'TLT_Adj Close',
 'TLT_EMA',
 'TLT_RSI',
 'TLT_MACD',
 'TLT_Signal',
 'TLT_MACD_minus_signal',
 'HYXF_Adj Close',
 'LQD_Adj Close',
 'bertopic_num',
 'd2v_umap_first_component',
 'd2v_umap_second_component',
 'USE_Score',
 'Return_Next_FOMC',
 'Return_Next_FOMC LQD',
 'Bull_Bear']

DF Pre Processing

In [9]:
df = df.tail(-1) 
df.set_index('Date', inplace=True)

#Imputing several values.
# Some metrics generate NAN or infinity values which cannot be handled by the ML algorithms below

# having to replace 2 infinity values
df.replace([np.inf, -np.inf], np.nan, inplace=True)
imputer = SimpleImputer(missing_values=np.nan, strategy='mean')

df[:] = imputer.fit_transform(df)

# Train Test Split

In [10]:
y = df[['Bull_Bear']] # separate out training and test data
X = df.drop(columns=['Bull_Bear','Return_Next_FOMC','SPY_Adj Close','TLT_Adj Close']) # removing these to reduce target contamination in X data

In [11]:
tss = TimeSeriesSplit(n_splits = 4,test_size=30)
for train_index, test_index in tss.split(X):
    X_train, X_test = X.iloc[train_index, :], X.iloc[test_index,:]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]



# A/B Testing Filtering

In [12]:
# Filter out NLP metrics for A/B Testing    
X_train_base = X_train.loc[:, ~X_train.columns.isin(NLP_metric)]
X_test_base = X_test.loc[:, ~X_test.columns.isin(NLP_metric)]

## Classifiers

Logistic Regression

In [13]:
#Data must be Scaled for Logistic Regression 

#Values without NLP data
scaler = preprocessing.StandardScaler().fit(X_train_base.values)
X_base_train_scaled = scaler.transform(X_train_base.values)
X_base_test_scaled = scaler.transform(X_test_base.values)

#Values with NLP data
scaler = preprocessing.StandardScaler().fit(X_train.values)
X_train_scaled = scaler.transform(X_train.values)
X_test_scaled = scaler.transform(X_test.values)


Logistic Regression Grid Search Parameters

In [14]:
reg_grid = {
    'solver': ['newton-cg','lbfgs', 'liblinear'],
    'penalty': ['none','l1', 'l2','elasticnet'],
    'C': loguniform(1e-5, 100)
}

In [15]:
def log_reg_base(reg_grid,X_base_train_scaled,X_base_test_scaled,y_test):
    clf = []
    clf = LogisticRegression(random_state=42)
    reg_cv = RandomizedSearchCV(clf,reg_grid, cv=10,n_iter=100,scoring='roc_auc',verbose=5,n_jobs=-1)
    _ = reg_cv.fit(X_base_train_scaled, np.ravel(y_train))
    reg_cv.best_params_
    clf = LogisticRegression(**reg_cv.best_params_,random_state=42)
    clf.fit(X_base_train_scaled, np.ravel(y_train))
    yhat = clf.predict(X_base_test_scaled)
    acc_reg = accuracy_score(y_test,yhat)
    print('Accuracy')
    print(acc_reg)
    return acc_reg

In [16]:
def log_reg_NLP(reg_grid,X_train_scaled,X_test_scaled,y_test):
    clf = []
    clf = LogisticRegression(random_state=42)
    reg_cv = RandomizedSearchCV(clf,reg_grid, cv=10,n_iter=100,scoring='roc_auc',verbose=5,n_jobs=-1)
    _ = reg_cv.fit(X_train_scaled, np.ravel(y_train))
    reg_cv.best_params_
    clf = LogisticRegression(**reg_cv.best_params_,random_state=42)
    clf.fit(X_train_scaled, np.ravel(y_train))
    yhat = clf.predict(X_test_scaled)
    acc_reg = accuracy_score(y_test,yhat)
    print('Accuracy With NLP ')
    print(acc_reg)
    return acc_reg

# Logistic Regression Monte Carlo Simulation

In [17]:

log_reg_vec = []
log_reg_NLP_vec =[]


for i in range(0,10):
    print('\n')
    print('***** Iteration ***** ',i+1)
    
    # Base Monte Carlo
    log_reg_score = log_reg_base(reg_grid,X_base_train_scaled,X_base_test_scaled,y_test)
    log_reg_vec.append(log_reg_score)

    #NLP Monte Carlo
    log_reg_NLP_score = log_reg_NLP(reg_grid,X_train_scaled,X_test_scaled,y_test)
    log_reg_NLP_vec.append(log_reg_NLP_score)





***** Iteration *****  1
Fitting 10 folds for each of 100 candidates, totalling 1000 fits
Accuracy
0.5333333333333333
Fitting 10 folds for each of 100 candidates, totalling 1000 fits
Accuracy With NLP 
0.7


***** Iteration *****  2
Fitting 10 folds for each of 100 candidates, totalling 1000 fits
Accuracy
0.6666666666666666
Fitting 10 folds for each of 100 candidates, totalling 1000 fits
Accuracy With NLP 
0.7


***** Iteration *****  3
Fitting 10 folds for each of 100 candidates, totalling 1000 fits
Accuracy
0.7
Fitting 10 folds for each of 100 candidates, totalling 1000 fits
Accuracy With NLP 
0.6666666666666666


***** Iteration *****  4
Fitting 10 folds for each of 100 candidates, totalling 1000 fits
Accuracy
0.6666666666666666
Fitting 10 folds for each of 100 candidates, totalling 1000 fits
Accuracy With NLP 
0.7


***** Iteration *****  5
Fitting 10 folds for each of 100 candidates, totalling 1000 fits
Accuracy
0.6666666666666666
Fitting 10 folds for each of 100 candidates, tot

In [18]:
# Average Accuracy Scores
LR_base_AVG = np.mean(log_reg_vec)
LR_NLP_AVG = np.mean(log_reg_NLP_vec)
print('Base Logistic Regression Accuracy: ',LR_base_AVG)
print('NLP Logistic Regression Accuracy: ',LR_NLP_AVG)

Base Logistic Regression Accuracy:  0.6466666666666667
NLP Logistic Regression Accuracy:  0.6833333333333333


## Random Forest

Set up Grid Search

In [19]:
n_estimators = [int(x) for x in np.linspace(start = 100, stop = 2000, num = 10)]
# Number of features to consider at every split
criterion = ['gini', 'entropy']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(1, 20, num = 20)]
max_depth.append(None)
# Minimum number of samples required to split a node
min_samples_split = [2, 5, 7]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4]
# Method of selecting samples for training each tree
bootstrap = [True, False]
# Create the random grid

random_grid = {'n_estimators': n_estimators,
               'criterion': criterion,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

In [20]:
def RF_base(random_grid,X_train_base,X_test_base,y_train,y_test):    

    clf_RF = RandomForestClassifier(random_state=42)
    rf_random2 = RandomizedSearchCV(estimator = clf_RF, param_distributions = random_grid, n_iter = 10, 
                                cv = 10, verbose=2, n_jobs = -1)
    rf_random2.fit(X_train_base, y_train.values.ravel())
    clf_RF = RandomForestClassifier(**rf_random2.best_params_,random_state=42)
    clf_RF.fit(X_train_base, y_train.values.ravel())
    yhat = clf_RF.predict(X_test_base)
    acc_RF = accuracy_score(y_test, yhat)

    print('Accuracy')
    print(acc_RF)


    return acc_RF


In [21]:
def RF_NLP(random_grid,X_train,X_test,y_train,y_test):    

    clf_RF = RandomForestClassifier(random_state=42)
    rf_random2 = RandomizedSearchCV(estimator = clf_RF, param_distributions = random_grid, n_iter = 10, 
                                cv = 10, verbose=2, n_jobs = -1)
    rf_random2.fit(X_train, y_train.values.ravel())
    clf_RF = RandomForestClassifier(**rf_random2.best_params_,random_state=42)
    clf_RF.fit(X_train, y_train.values.ravel())
    yhat = clf_RF.predict(X_test)
    acc_RF = accuracy_score(y_test, yhat)

    print('Accuracy With NLP')
    print(acc_RF)


    return acc_RF

# Random Forest Monte Carlo Simulation

In [22]:
RF_vec = []
RF_NLP_vec =[]


for i in range(0,5):
    print('\n')
    print('***** Iteration ***** ',i+1)
    # Base Monte Carlo
    RF_base_score = RF_base(random_grid,X_train_base,X_test_base,y_train,y_test)
    RF_vec.append(RF_base_score)

    #NLP Monte Carlo
    RF_NLP_score = RF_NLP(random_grid,X_train,X_test,y_train,y_test)
    RF_NLP_vec.append(RF_NLP_score)




***** Iteration *****  1
Fitting 10 folds for each of 10 candidates, totalling 100 fits
Accuracy
0.7666666666666667
Fitting 10 folds for each of 10 candidates, totalling 100 fits
Accuracy With NLP
0.7333333333333333


***** Iteration *****  2
Fitting 10 folds for each of 10 candidates, totalling 100 fits
Accuracy
0.8
Fitting 10 folds for each of 10 candidates, totalling 100 fits
Accuracy With NLP
0.7666666666666667


***** Iteration *****  3
Fitting 10 folds for each of 10 candidates, totalling 100 fits
Accuracy
0.7
Fitting 10 folds for each of 10 candidates, totalling 100 fits
Accuracy With NLP
0.7666666666666667


***** Iteration *****  4
Fitting 10 folds for each of 10 candidates, totalling 100 fits
Accuracy
0.7666666666666667
Fitting 10 folds for each of 10 candidates, totalling 100 fits
Accuracy With NLP
0.7666666666666667


***** Iteration *****  5
Fitting 10 folds for each of 10 candidates, totalling 100 fits
Accuracy
0.7
Fitting 10 folds for each of 10 candidates, totalling 1

In [23]:
RF_base_AVG = np.mean(RF_vec)
RF_NLP_vec = np.mean(RF_NLP_vec)

print('Base Random Forest Accuracy: ',RF_base_AVG)
print('NLP Random Forest Accuracy: ',RF_NLP_vec)

Base Random Forest Accuracy:  0.7466666666666667
NLP Random Forest Accuracy:  0.7533333333333333


## XGBoost

In [24]:
param_grid = {
    "max_depth": [1,2,3, 4, 5, 7],
    "learning_rate": [0.1, 0.01, 0.05,0.1,0.15],
    "gamma": [0, 0.25, 1,1.25,1.5,1.75],
    "reg_lambda": [0, 1, 10,50,100,110,120],
    "scale_pos_weight": [1, 3, 5,7,9,11],
    "subsample": [0.8],
    "colsample_bytree": [0.5],
}

In [25]:
def XG_NLP(param_grid,X_train,X_test,y_train,y_test):

    #Optimizing through grid search
    xgb_cl = xgb.XGBClassifier(objective="binary:logistic",random_state=42)
    grid_cv = RandomizedSearchCV(xgb_cl,param_grid, cv=10,n_iter=100,scoring='roc_auc',verbose=5,n_jobs=-1)
    _ = grid_cv.fit(X_train, y_train)

    # Rebuild optimized clasifier
    xgb_cl = xgb.XGBClassifier(**grid_cv.best_params_,objective="binary:logistic",random_state=42)
    xgb_cl.fit(X_train, y_train)
    yhat = xgb_cl.predict(X_test)
    acc_nlp = accuracy_score(y_test,yhat)
    print('Accuracy With NLP')
    print(acc_nlp)


    return (acc_nlp, grid_cv.best_params_)

In [26]:
def XG_Base(param_grid,X_train_base,X_test_base,y_train,y_test):

    xgb_cl = xgb.XGBClassifier(objective="binary:logistic",random_state=42)
 
    grid_cv2 = RandomizedSearchCV(xgb_cl,param_grid, cv=10,n_iter=100,scoring='roc_auc',verbose=5,n_jobs=-1)
    _ = grid_cv2.fit(X_train_base, y_train) # base is used instead of X_train

    xgb_base2 = xgb.XGBClassifier(**grid_cv2.best_params_,objective="binary:logistic",random_state=42)

    xgb_base2.fit(X_train_base, y_train)
    yhat_base = xgb_base2.predict(X_test_base)
    acc_base = accuracy_score(y_test,yhat_base)

    print('Accuracy')
    print(acc_base)

    return (acc_base,grid_cv2.best_params_)

## XGBoost Monte Carlo

In [41]:
XG_vec = []
XG_NLP_vec =[]
XG_base_top_score = -1
XG_top_score = -1
for i in range(0,10):
    print('\n')
    print('***** Iteration ***** ',i+1)
    # Base Monte Carlo
    XG_base_score,base_best_parameters = XG_Base(param_grid,X_train_base,X_test_base,y_train,y_test)
    XG_vec.append(XG_base_score)
    if XG_base_score > XG_base_top_score:
        best_iter_base = base_best_parameters


    #NLP Monte Carlo
    XG_NLP_score,best_parameters = XG_NLP(param_grid,X_train,X_test,y_train,y_test)
    XG_NLP_vec.append(XG_NLP_score)
    if XG_NLP_score > XG_top_score:
        best_iter = best_parameters
    
    



***** Iteration *****  1
Fitting 10 folds for each of 100 candidates, totalling 1000 fits
Accuracy
0.7333333333333333
Fitting 10 folds for each of 100 candidates, totalling 1000 fits
Accuracy With NLP
0.7333333333333333


***** Iteration *****  2
Fitting 10 folds for each of 100 candidates, totalling 1000 fits
Accuracy
0.6
Fitting 10 folds for each of 100 candidates, totalling 1000 fits
Accuracy With NLP
0.7333333333333333


***** Iteration *****  3
Fitting 10 folds for each of 100 candidates, totalling 1000 fits
Accuracy
0.7333333333333333
Fitting 10 folds for each of 100 candidates, totalling 1000 fits
Accuracy With NLP
0.7333333333333333


***** Iteration *****  4
Fitting 10 folds for each of 100 candidates, totalling 1000 fits
Accuracy
0.6
Fitting 10 folds for each of 100 candidates, totalling 1000 fits
Accuracy With NLP
0.7333333333333333


***** Iteration *****  5
Fitting 10 folds for each of 100 candidates, totalling 1000 fits
Accuracy
0.7333333333333333
Fitting 10 folds for e

In [42]:
XG_base_AVG = np.mean(XG_vec)
XG_NLP_AVG = np.mean(XG_NLP_vec)

print('Base XGBoost Accuracy: ',XG_base_AVG)
print('NLP XGBoost Accuracy: ',XG_NLP_AVG)

Base XGBoost Accuracy:  0.6733333333333332
NLP XGBoost Accuracy:  0.7333333333333333


# Final Model Best Params


In [43]:
xgb_cl = xgb.XGBClassifier(objective="binary:logistic",random_state=42)
grid_cv2 = RandomizedSearchCV(xgb_cl,param_grid, cv=10,n_iter=100,scoring='roc_auc',verbose=5,n_jobs=-1)
_ = grid_cv2.fit(X_train_base, y_train) # base is used instead of X_train
xgb_base2 = xgb.XGBClassifier(**best_iter_base,objective="binary:logistic",random_state=42)

xgb_base2.fit(X_train_base, y_train)
yhat_base = xgb_base2.predict(X_test_base)
acc_base = accuracy_score(y_test,yhat_base)
acc_base


Fitting 10 folds for each of 100 candidates, totalling 1000 fits


0.6333333333333333

In [44]:
xgb_cl = xgb.XGBClassifier(objective="binary:logistic",random_state=42)
grid_cv2 = RandomizedSearchCV(xgb_cl,param_grid, cv=10,n_iter=100,scoring='roc_auc',verbose=5,n_jobs=-1)
_ = grid_cv2.fit(X_train_base, y_train) # base is used instead of X_train
xgb_base2 = xgb.XGBClassifier(**best_iter,objective="binary:logistic",random_state=42)

xgb_base2.fit(X_train, y_train)
yhat = xgb_base2.predict(X_test)
acc_base = accuracy_score(y_test,yhat)
acc_base

Fitting 10 folds for each of 100 candidates, totalling 1000 fits


0.7333333333333333

In [54]:
comp_df = y_test.copy()
comp_df['Prediction'] = yhat
comp_df['Base_Prediction']=yhat_base
comp_df.to_csv('Excel_data/Predictions_random.csv')

# Print out the List of All Predictions. 

Prediction is the model with NLP Included and Base is the non NLP version 

In [46]:
comp_df

,Bull_Bear,Prediction,Base_Prediction
Date,,,
2019-10-30,1.0,1,1
2019-12-11,0.0,0,1
2020-01-29,0.0,0,1
2020-03-15,0.0,1,1
2020-04-29,1.0,1,1
2020-06-10,0.0,0,0
2020-07-29,1.0,0,1
2020-09-16,0.0,0,1
2020-11-05,1.0,1,1


Quick Tabulation of Missed Predictions

In [49]:
comp_df
NLP_counter = comp_df[comp_df['Prediction']!=comp_df['Bull_Bear']].count()
NLP_counter

Bull_Bear          8
Prediction         8
Base_Prediction    8
dtype: int64

# Monte Carlo Analysis of Classifiers

In [51]:
# These values were batch processed with the code above and saved to a csv file which is now read in as nlp_mc_Final

nlp_mc = pd.read_csv('Excel_data/nlp_mc_Final.csv')
nlp_mc = pd.DataFrame(nlp_mc)
nlp_mc = nlp_mc.rename(columns={'Unnamed: 0':'Metric'})
nlp_mc['Logistic Regression'] =nlp_mc['Logistic Regression']*100
nlp_mc['Random Forest'] =nlp_mc['Random Forest']*100
nlp_mc['XGBoost'] =nlp_mc['XGBoost']*100
nlp_mc

,Metric,Logistic Regression,Random Forest,XGBoost
0,No NLP,66.33,63.33,73.66
1,Transformer Sentiment,58.00,66.66,73.00
2,Cosine Similarity,67.66,66.66,68.66
3,USE Score,65.99,66.66,75.66
4,Bert Topic Modeling,66.33,70.00,75.00
5,LDA max Value,66.33,69.33,69.00
6,LDA Mosaic,67.99,68.66,73.00
7,Doc2Vec Mean,68.33,71.33,71.66
8,Doc2Vec Mosaic,63.00,71.33,73.33
9,D2V UMAP 1st Comp,62.66,69.33,71.00


# A-B Testing Plot

In [52]:
import plotly.graph_objects as go

metric = nlp_mc.Metric.to_list()

fig = go.Figure()
fig.add_trace(go.Bar(
    x=metric,
    y=nlp_mc['Logistic Regression'],
    name='Logistic Regression',
    marker_color='lightsalmon',
    texttemplate="%{y:0.2f}",
    text=nlp_mc['XGBoost'].round(2),
    #width=0.3
    
))
fig.add_trace(go.Bar(
    x=metric,
    y=nlp_mc['Random Forest'],
    name='Random Forest',
    marker_color='indianred',
    texttemplate="%{y:0.2f}",
    text=nlp_mc['XGBoost'].round(2),
    #width=0.3

))
fig.add_trace(go.Bar(
    x=metric,
    y=nlp_mc['XGBoost'],
    name='XGBoost',
    marker_color='steelblue',
    selectedpoints=None,
    textposition='auto',
    texttemplate="%{y:0.1f}",
    text=nlp_mc['XGBoost'].round(2),
    #width=0.3
))

fig.add_annotation(x=13.27, y=71,
            text="Best",
            showarrow=False,
            font = dict(size = 9),
            yshift=40)


fig.update_layout(barmode='group', xaxis_tickangle=-45)
fig.update_layout(title=dict(text="Average Model Prediction Accuracy By NLP Metric",font=dict(size=27),x=0.35,y=0.85,xref="paper"),yaxis_title=("Accuracy (%)"),xaxis_title="NLP Method")
fig.update_layout(yaxis_title="Accuracy (%)",font=dict(size=12))
fig.update_layout(yaxis_range=[50,85])
fig.update_layout(xaxis_title="NLP Metric",font=dict(size=14))
fig.update_layout(
    margin=dict(l=0, r=0, t=100, b=0),
)

fig.update_layout(legend=dict(
    yanchor="top",
    y=1.4,
    xanchor="left",
    x=.80
))


fig.show()